# Case Final
## Análise de Transações PIX
CRISP-DM - https://www.escoladnc.com.br/blog/data-science/metodologia-crisp-dm/
### Objetivos
- Limpar e pré-processar os dados das transações PIX
- Analisar padrões de uso do PIX, tais como os canais mais utilizados e os valores de transação mais comuns
- Use o PySpark MLlib para treinar e avaliar um modelo de detecção de fraude
- Avaliar o desempenho do modelo e fazer recomendações para melhorias futuras

### Dados

O conjunto de dados inclui as seguintes informações para cada transação:
- Detalhes da transação: valor, tempo, remetente e receptor CPF/CNPJ, tipo
- Etiqueta de fraude: uma variável binária que indica se a transação foi fraudulenta (1) ou não (0)

### Tarefas
- Normalização dos dados:
  - O dataset que você lerá está em formato json.
  ```json
  {
      "id_transacao": inteiro,
      "valor": texto,
      "remetente": {
          "nome": texto,
          "banco": texto,
          "tipo": texto
      },
      "destinatario": {
          "nome": texto,
          "banco":texto,
          "tipo": texto
      },          
      "categoria":texto,
      "chave_pix":texto,
      "transaction_date":texto,
      "fraude":inteiro,
  }
    ```
  - Faça sua transformação para formato colunar
- Análise Exploratória de Dados: Use o PySpark para analisar padrões de uso do PIX:
  - chaves pix mais usadas;
  - os valores de transação mais comuns;
  - distribuição dos valores de transação por hora e dia;
  - quais bancos receberam mais transferências por dia;
  - para qual tipo de pessoa (PF ou PJ) foram realizadas mais transações
- Engenharia de Recursos: Apresentar novas características que podem ser úteis para a detecção de fraudes, tais como o número de transações feitas pelo mesmo remetente em um período de tempo específico.
- Modelagem: Use o PySpark MLlib para treinar e detectar possíveis transações que contenham fraude.

### Observação
É importante notar que este é um caso simplificado, e em cenários do mundo real você teria que lidar com dados mais complexos, usar técnicas mais avançadas como métodos de conjuntos e considerar o conhecimento de domínio, bem como leis e regulamentos das instituições financeiras no Brasil.


### Observação II
Não existe resposta 100% correta. É necessário que você use seu pensamento crítico para definir as melhores métricas e análises para o caso.

# Entendimento do Negócio
Você trabalha em um banco e o principal meio de pagamento utilizado no seu banco é o Pix.

Através da base de transações do pix o banco deseja entender qual é o perfil dos clientes que utilizam o pix, além de verificar possíveis transações que tenham fraude. Porém, eles tem um cliente específico que tem um relacionamento muito bom para o banco, por isso, você recebeu a base de transações de cliente dos últimos 2 anos e precisa a partir dela criar um relatório contendo as principais características das transações.


Então, resumindo, temos dois principais objetivos para esse case:
1. Obter valor a partir dos dados
  - Para qual banco esse cliente mais transfere?
  - Qual é a média de transferências por período que esse cliente faz?
  - Baseando-se no valor das transferências, poderia dar um aumento de crédito?
  - Para o que esse cliente mais usa as transferências?
2. Executar um algoritmo de machine learning que identifique possíveis transações com fraude.
3. Pós Processamento
  - Defina ao mínimo cinco métricas de qualidade para seus dados
  - Explique se os seus dados estão com uma boa qualidade

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparação do Ambiente de Desenvolvimento

In [2]:
# Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=7c1c83f5853df896917f41e2ac85ff7e118ee4dd32742a69cdbaf50c33d65763
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Data Undesrtanting

Primeiramente, devemos entender tudo sobre a fonte dos dados
- Como o dado chega até nós?
- Qual formato virá?
- Aonde o processamento será executado (AWS EMR, Cluster On-Premise)?
- De quanto em quanto tempo eu preciso gerar esse relatório (mensal, diário, near-real time)?

```json
{
  "id_transacao": inteiro,
  "valor": texto,
  "remetente": {
      "nome": texto,
      "banco": texto,
      "tipo": texto
  },
  "destinatario": {
      "nome": texto,
      "banco":texto,
      "tipo": texto
  },
  "categoria": texto,
  "transaction_date":texto,
  "chave_pix":texto,
  "fraude":inteiro,
}
```



# Preparação dos Dados
Agora é hora de começar a preparar os dados de acordo com as suas necessidades.

In [4]:
# Iniciar a sessão spark
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
      .config('spark.ui.port', '4050')
      .appName("CaseFinal")
      .getOrCreate()
)


In [6]:
from pyspark.sql.types import *


schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])


schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])




df = spark.read.json('/content/drive/MyDrive/Colab Notebooks/Big data/Case final/Aula 26/case_final.json', schema= schema_base_pix, timestampFormat= 'yyyy-MM-dd HH:mm:ss')

In [7]:
df.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)



In [8]:
df.show()

+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|        1000|            588.08|{Jonathan Gonsalv...|{Calebe Melo, Cai...|aleatoria|       outros|2021-07-16 05:00:55|     0|
|        1001|           80682.5|{Jonathan Gonsalv...|{Davi Lucas Perei...|  celular|transferencia|2022-04-20 12:34:01|     1|
|        1002|             549.9|{Jonathan Gonsalv...|{Sabrina Castro, ...|      cpf|        lazer|2022-07-10 16:51:34|     0|
|        1003|             90.83|{Jonathan Gonsalv...|{Francisco da Con...|aleatoria|   transporte|2022-10-20 10:57:36|     0|
|        1004|13272.619999999999|{Jonathan Gonsalv...|{Isabelly Ferreir...|    email|transferencia|2021-04-06 2

Criando o Flaten

In [13]:
from pyspark.sql.functions import col

df_flaten = df.withColumns({
    'remetente_nome':  col('remetente').getField('nome'),
    'remetente_banco':  col('remetente').getField('banco'),
    'remetente_tipo':  col('remetente').getField('tipo'),
    'destinatario_nome':  col('destinatario').getField('nome'),
    'destinatario_banco':  col('destinatario').getField('banco'),
    'destinatario_tipo':  col('destinatario').getField('tipo'),
}).drop('remetente', 'destinatario')

In [14]:
df_flaten.show()

+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|    remetente_nome|remetente_banco|remetente_tipo|   destinatario_nome|destinatario_banco|destinatario_tipo|
+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|Jonathan Gonsalves|            BTG|            PF|         Calebe Melo|             Caixa|               PF|
|        1001|           80682.5|  celular|transferencia|2022-04-20 12:34:01|     1|Jonathan Gonsalves|            BTG|            PF|  Davi Lucas Pereira|             Caixa|               PJ|
|        1002|             549.9|  

In [16]:
df_flaten.describe().show()

# Media de fraude mostra que aproximadamente 15% dos registros são fraude

+-------+-----------------+------------------+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|summary|     id_transacao|             valor|chave_pix|  categoria|   transaction_date|            fraude|    remetente_nome|remetente_banco|remetente_tipo|destinatario_nome|destinatario_banco|destinatario_tipo|
+-------+-----------------+------------------+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|  count|           100000|            100000|   100000|     100000|             100000|            100000|            100000|         100000|        100000|           100000|            100000|           100000|
|   mean|          50999.5|10303.358732200059|     NULL|       NULL|               NULL|           0.15367|              NULL|           NULL|      

# Modelagem
Aqui você encontrará utilidade para os dados levantados.

Aqui será onde teremos insights e, a partir desses, novos conhecimentos sobre o business (se tudo até aqui foi feito corretamente).


- Para qual banco esse cliente mais transfere?
- Qual é a média de transferências por período que esse cliente faz?
- Baseando-se no valor das transferências, poderia dar um aumento de crédito?
- Para o que esse cliente mais usa as transferências?
- Executar um algoritmo de machine learning que identifique possíveis transações com fraude.


## Modelo de Predição de Fraudes

# Avaliação do Modelo
Será que seu modelo atinge todas as necessidades que foram definidas inicialmente? (e.g. pessoa em cima da bicicleta muda o resultado final)



# Deployment
Apresente o relatório com os resultados obtidos.




  